In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms, models
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from skimage import io
import os
from PIL import Image
import numpy as np
from torch.optim import lr_scheduler

In [2]:
path = '/home/jeet/WEBEmo/category.txt'

with open(path, 'r') as f:
    content = f.readlines()
    
content = [elem.strip('\n').split(',') for elem in content]   

for elem in content:
    if elem[2] == '+':
        elem[2] = 'positive'
    else:
        elem[2] = 'negative'
        
print (content)

[['affection', 'love', 'positive'], ['cheerfullness', 'joy', 'positive'], ['confusion', 'confusion', 'negative'], ['contentment', 'joy', 'positive'], ['disappointment', 'sadness', 'negative'], ['disgust', 'anger', 'negative'], ['enthrallment', 'joy', 'positive'], ['envy', 'anger', 'negative'], ['exasperation', 'anger', 'negative'], ['gratitude', 'love', 'positive'], ['horror', 'fear', 'negative'], ['irritabilty', 'anger', 'negative'], ['lust', 'love', 'positive'], ['neglect', 'sadness', 'negative'], ['nervousness', 'fear', 'negative'], ['optimism', 'joy', 'positive'], ['pride', 'joy', 'positive'], ['rage', 'anger', 'negative'], ['relief', 'joy', 'positive'], ['sadness', 'sadness', 'negative'], ['shame', 'sadness', 'negative'], ['suffering', 'sadness', 'negative'], ['surprise', 'surprise', 'positive'], ['sympathy', 'sadness', 'negative'], ['zest', 'joy', 'positive']]


In [3]:
level1 = dict()
level2 = dict()

for i, elem in enumerate(content):
    if elem[2] not in level1.keys():
        level1[elem[2]] = []
    level1[elem[2]].append(i)
    
    if elem[1] not in level2.keys():
        level2[elem[1]] = []
        
    level2[elem[1]].append(i)
    
print (level1)
print (level2)

{'positive': [0, 1, 3, 6, 9, 12, 15, 16, 18, 22, 24], 'negative': [2, 4, 5, 7, 8, 10, 11, 13, 14, 17, 19, 20, 21, 23]}
{'love': [0, 9, 12], 'joy': [1, 3, 6, 15, 16, 18, 24], 'confusion': [2], 'sadness': [4, 13, 19, 20, 21, 23], 'anger': [5, 7, 8, 11, 17], 'fear': [10, 14], 'surprise': [22]}


In [4]:
# Function to extract the label of the folder 
def get_key(label_dict, val):
    for key, val_list in label_dict.items():
        if val in val_list:
            return key
        
# Function to make the dataset. Returns list of tuple (path, label) for the image
def make_dataset(root_dir, label_dict, class_to_idx):
    images = []
    for target in sorted(os.listdir(root_dir)):
        d = os.path.join(root_dir, target)
        
        try :
            int(target)
        except:
            continue
        
        label = get_key(label_dict, int(target))
        label = class_to_idx[label]

        for root, _, fnames in sorted(os.walk(d)):
            for fname in sorted(fnames):
                path = os.path.join(root, fname)
                item = (path, label)
                images.append(item)

    return images

# Helper function to load the images given the path of the image
def pil_loader(path):
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')
    
# Attribute of the class Level1ImageDataSet    
def find_classes(root_dir, label_dict):
    classes = []
    
    for label_dir in sorted(os.listdir(root_dir)):
        try:
            int(label_dir)
            classes.append(get_key(label_dict, int(label_dir)))
        except:
            continue
    
    classes = list(set(classes))
    
    class_to_idx = {classes[i]: i for i in range(len(classes))}
    return classes, class_to_idx

In [5]:
class CustomImageDataset(Dataset):
    
    def __init__(self, root_dir, label_dict, transform=None):
        super(CustomImageDataset, self).__init__()
        
        classes, class_to_idx = find_classes(root_dir, label_dict)
        samples = make_dataset(root_dir, label_dict, class_to_idx)
        
        self.root_dir = root_dir
        self.transform = transform
        self.label_dict = label_dict
        
        self.samples = samples
        self.classes = classes
        self.class_to_idx = class_to_idx
        
    def __len__(self):
        return (len(self.samples))
    
    def __getitem__(self, index):
        
        path, label = self.samples[index]
        sample = pil_loader(path)
        
        if self.transform is not None:
            sample = self.transform(sample)
        
        return sample, label
    
    def __repr__(self):
        fmt_str = 'Dataset ' + self.__class__.__name__ + '\n'
        fmt_str += '    Number of datapoints: {}\n'.format(self.__len__())
        fmt_str += '    Root Location: {}\n'.format(self.root_dir)
        tmp = '    Transforms (if any): '
        fmt_str += '{0}{1}\n'.format(tmp, self.transform.__repr__().replace('\n', '\n' + ' ' * len(tmp)))
        return fmt_str

In [6]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [7]:
data_dir = '/home/jeet/WEBEmo/'
BATCH_SIZE = 64

dset_l1 = {x: CustomImageDataset(os.path.join(data_dir, x), level1, data_transforms[x]) for x in ['train','test']}
dset_l2 = {x: CustomImageDataset(os.path.join(data_dir, x), level2, data_transforms[x]) for x in ['train','test']}

dset_loader_l1 = {x: DataLoader(dset_l1[x], batch_size=BATCH_SIZE, shuffle=True, num_workers=16) for x in ['train', 'test']}
dset_loader_l2 = {x: DataLoader(dset_l2[x], batch_size=BATCH_SIZE, shuffle=True, num_workers=16) for x in ['train', 'test']}

dset_sizes = {x: len(dset_l1[x]) for x in ['train', 'test']}
dset_sizes = {x: len(dset_l2[x]) for x in ['train', 'test']}

dset_classes_l1 = dset_l1['train'].classes
dset_classes_l2 = dset_l2['train'].classes

In [8]:
def train(model, dataloader, criterion, optimizer, scheduler, num_epochs = 20):
    
    for epoch in range(num_epochs):
        scheduler.step()
        model.train()
        
        best_acc = 0.0
        epoch_acc = 0.0
        epoch_loss = 0.0
        train_loss = 0.0
        train_acc = 0.0
        
        for i, (images, labels) in enumerate(dataloader['train']):
            
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
        
            optimizer.zero_grad()
            outputs = model(images)
            
            _, preds = torch.max(outputs, 1)
            
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * images.size(0)
            train_acc += torch.sum(preds == labels.data)
            
            if i % 100 == 0:
                try:
                    avg = train_acc.double() / (i * BATCH_SIZE)
                    print ("Average correctly classified images till {} batches: {}".format(i, avg))
                except:
                    continue
            
        epoch_loss = train_loss / (len(dataloader['train']) * BATCH_SIZE)
        epoch_acc = train_acc.double() / (len(dataloader['train']) * BATCH_SIZE)
        
        print ("Epoch: {}, Epoch_Accuracy: {:.2f}, Epoch_loss: {:.4f}".format(epoch, epoch_acc, epoch_loss))
        
        # Evaluate on the test set
        test_acc = test(model, dataloader)

#       Save the model weights if the test acc is greater than our current best
        if test_acc > best_acc:
            best_model_wt = model.state_dict()
            print("Chekcpoint updated")
            best_acc = test_acc

        # Print the metrics
        print("Test Accuracy: {:.4f}, Best Accuracy: {:.4f}".format(test_acc, best_acc))
        
#   Save the best model weights
    torch.save(best_model_wt, "Resnet50_Transfer_Learning.model")

In [9]:
def test(model, dataloader):
    model.eval()
    test_acc = 0.0
    
    for i, (images, labels) in enumerate(dataloader['test']):

        images = Variable(images.cuda())
        labels = Variable(labels.cuda())

        # Predict classes using images from the test set
        outputs = model(images)
        _, prediction = torch.max(outputs.data, 1)
        
        test_acc += torch.sum(prediction == labels.data)

    # Compute the average acc and loss over all test images
    test_acc = test_acc.double() / (len(dataloader['test']) * BATCH_SIZE)

    return test_acc

In [10]:
model_conv = models.resnet50(pretrained=True)

for params in model_conv.parameters():
    params.requires_grad=False
    
num_features = model_conv.fc.in_features
model_conv.fc = nn.Sequential(nn.Linear(num_features, 256),
                              nn.BatchNorm1d(256),
                              nn.ReLU(),
                              nn.Dropout(p = 0.5),
                              nn.Linear(256, 50),
                              nn.BatchNorm1d(50),
                              nn.ReLU(),
                              nn.Dropout(p = 0.5),
                              nn.Linear(50, 2))

model_conv.cuda()

criterion = nn.CrossEntropyLoss()

optimizer_conv = optim.Adam(model_conv.parameters(), lr=0.001, weight_decay=0.0001)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

In [11]:
print (model_conv)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [ ]:
train(model_conv, dset_loader_l1, criterion, optimizer_conv, exp_lr_scheduler, num_epochs=10)

Average correctly classified images till 0 batches: inf
Average correctly classified images till 100 batches: 0.59109375
Average correctly classified images till 200 batches: 0.61265625
Average correctly classified images till 300 batches: 0.6220833333333333
Average correctly classified images till 400 batches: 0.62546875


#### Training for the second hierarchy after removing the last linear layer and adding another linear layer of 6 dimension

In [15]:
for params in model_conv.parameters():
    params.requires_grad=False
    
model_conv.fc = nn.Sequential(*list(model_conv.fc.children())[:-4],
                               nn.BatchNorm1d(50),
                               nn.ReLU(),
                               nn.Dropout(p = 0.5),
                               nn.Linear(50,7))

print (model_conv)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [ ]:
train(model_conv, dset_loader_l2, criterion, optimizer_conv, exp_lr_scheduler, num_epochs=20)